# 🎥 파이썬으로 유튜브 영상 다운로드하기

이 노트북에서는 파이썬을 사용하여 유튜브 영상을 다운로드하는 방법을 학습합니다.

## 📋 목차
1. 라이브러리 설치 및 임포트
2. 단일 영상 다운로드
3. 여러 영상 일괄 다운로드
4. 특정 해상도로 다운로드
5. 오디오만 다운로드 (MP3)
6. 영상 정보 확인
7. 진행률 표시 다운로드

## ⚠️ 주의사항
- 다운로드한 영상은 개인적인 용도로만 사용하세요
- 저작권을 준수해주세요
- 안정적인 인터넷 연결이 필요합니다

## 1️⃣ 라이브러리 설치 및 임포트

먼저 필요한 라이브러리들을 설치하고 임포트합니다.
- `pytube`: 유튜브 영상 다운로드를 위한 기본 라이브러리
- `yt-dlp`: 더 안정적이고 기능이 많은 대안 라이브러리

In [ ]:
# 필요한 라이브러리 설치
# 처음 실행할 때만 실행하면 됩니다

!pip install pytube
!pip install yt-dlp

In [ ]:
# 필요한 라이브러리 임포트
import os
import sys
from pytube import YouTube
from yt_dlp import YoutubeDL
import time

print("✅ 모든 라이브러리가 성공적으로 임포트되었습니다!")
print(f"현재 작업 디렉토리: {os.getcwd()}")

## 2️⃣ 단일 영상 다운로드 (pytube 사용)

가장 기본적인 방법으로 하나의 유튜브 영상을 다운로드합니다.

In [ ]:
def download_youtube_video(video_url, download_path="./downloads"):
    """
    유튜브 영상을 다운로드하는 함수
    
    Parameters:
    video_url (str): 다운로드할 유튜브 영상 URL
    download_path (str): 다운로드할 폴더 경로 (기본값: './downloads')
    
    Returns:
    bool: 다운로드 성공 여부
    """
    try:
        print(f"🔗 URL 처리 중: {video_url}")
        
        # YouTube 객체 생성
        yt = YouTube(video_url)
        
        # 영상 정보 출력
        print(f"📺 제목: {yt.title}")
        print(f"👤 채널: {yt.author}")
        print(f"⏱️ 길이: {yt.length}초")
        
        # 가장 높은 해상도의 mp4 스트림 선택
        # progressive=True: 비디오와 오디오가 함께 있는 스트림
        video_stream = yt.streams.filter(
            progressive=True, 
            file_extension='mp4'
        ).order_by('resolution').desc().first()
        
        if video_stream is None:
            print("❌ 다운로드 가능한 MP4 스트림을 찾을 수 없습니다.")
            return False
        
        print(f"📹 선택된 해상도: {video_stream.resolution}")
        print(f"💾 파일 크기: {video_stream.filesize_mb:.1f}MB")
        
        # 다운로드 폴더가 없으면 생성
        if not os.path.exists(download_path):
            os.makedirs(download_path)
            print(f"📁 폴더 생성: {download_path}")
        
        # 영상 다운로드 시작
        print("⬇️ 다운로드 시작...")
        video_stream.download(download_path)
        print(f"✅ 다운로드 완료: {yt.title}")
        
        return True
        
    except Exception as e:
        print(f"❌ 오류 발생: {str(e)}")
        return False

# 함수 정의 완료
print("✅ download_youtube_video 함수가 정의되었습니다.")

In [ ]:
# 실제 다운로드 실행
# ⚠️ 아래 URL을 실제 유튜브 영상 URL로 변경해주세요

video_url = "https://www.youtube.com/watch?v=dQw4w9WgXcQ"  # 예시 URL (Rick Roll)

# 다운로드 실행
success = download_youtube_video(video_url)

if success:
    print("\n🎉 다운로드가 성공적으로 완료되었습니다!")
else:
    print("\n😞 다운로드에 실패했습니다. URL을 확인해주세요.")

## 3️⃣ 여러 영상 일괄 다운로드

여러 개의 유튜브 영상을 한 번에 다운로드하는 기능입니다.

In [ ]:
def download_multiple_videos(video_urls, download_path="./downloads"):
    """
    여러 유튜브 영상을 일괄 다운로드하는 함수
    
    Parameters:
    video_urls (list): 다운로드할 유튜브 영상 URL 리스트
    download_path (str): 다운로드할 폴더 경로
    
    Returns:
    dict: 다운로드 결과 통계
    """
    # 결과 통계를 위한 변수들
    total_videos = len(video_urls)
    successful_downloads = 0
    failed_downloads = 0
    
    print(f"📊 총 {total_videos}개의 영상을 다운로드합니다.\n")
    
    # 다운로드 폴더 생성
    if not os.path.exists(download_path):
        os.makedirs(download_path)
        print(f"📁 폴더 생성: {download_path}\n")
    
    # 각 URL에 대해 다운로드 실행
    for i, url in enumerate(video_urls, 1):
        print(f"\n{'='*50}")
        print(f"📹 [{i}/{total_videos}] 영상 처리 중...")
        print(f"{'='*50}")
        
        try:
            # YouTube 객체 생성
            yt = YouTube(url)
            
            # 영상 정보 출력
            print(f"📺 제목: {yt.title}")
            print(f"👤 채널: {yt.author}")
            
            # 최고 해상도의 mp4 스트림 선택
            video_stream = yt.streams.filter(
                progressive=True, 
                file_extension='mp4'
            ).order_by('resolution').desc().first()
            
            if video_stream is None:
                print("❌ 다운로드 가능한 스트림을 찾을 수 없습니다.")
                failed_downloads += 1
                continue
            
            print(f"📹 해상도: {video_stream.resolution}")
            print(f"💾 크기: {video_stream.filesize_mb:.1f}MB")
            
            # 영상 다운로드
            print("⬇️ 다운로드 중...")
            video_stream.download(download_path)
            print(f"✅ 다운로드 완료!")
            
            successful_downloads += 1
            
            # 다음 다운로드 전 잠시 대기 (서버 부하 방지)
            if i < total_videos:
                print("⏳ 잠시 대기 중... (3초)")
                time.sleep(3)
            
        except Exception as e:
            print(f"❌ 오류 발생: {str(e)}")
            failed_downloads += 1
    
    # 최종 결과 출력
    print(f"\n{'='*50}")
    print("📊 다운로드 완료 통계")
    print(f"{'='*50}")
    print(f"✅ 성공: {successful_downloads}개")
    print(f"❌ 실패: {failed_downloads}개")
    print(f"📊 성공률: {(successful_downloads/total_videos)*100:.1f}%")
    
    return {
        'total': total_videos,
        'successful': successful_downloads,
        'failed': failed_downloads,
        'success_rate': (successful_downloads/total_videos)*100
    }

print("✅ download_multiple_videos 함수가 정의되었습니다.")

In [ ]:
# 여러 영상 다운로드 실행 예시
# ⚠️ 아래 URL들을 실제 유튜브 영상 URL로 변경해주세요

video_list = [
    "https://www.youtube.com/watch?v=dQw4w9WgXcQ",  # 예시 URL 1
    "https://www.youtube.com/watch?v=9bZkp7q19f0",  # 예시 URL 2
    # 더 많은 URL을 추가할 수 있습니다
]

# 다운로드 실행
results = download_multiple_videos(video_list, "./batch_downloads")

print(f"\n🎯 최종 결과: {results['successful']}/{results['total']} 성공")

## 4️⃣ 특정 해상도로 다운로드

원하는 해상도를 지정하여 영상을 다운로드할 수 있습니다.

In [ ]:
def download_with_resolution(video_url, resolution="720p", download_path="./downloads"):
    """
    특정 해상도로 유튜브 영상을 다운로드하는 함수
    
    Parameters:
    video_url (str): 다운로드할 유튜브 영상 URL
    resolution (str): 원하는 해상도 (예: '720p', '480p', '360p', '1080p')
    download_path (str): 다운로드할 폴더 경로
    
    Returns:
    bool: 다운로드 성공 여부
    """
    try:
        print(f"🎯 목표 해상도: {resolution}")
        print(f"🔗 URL 처리 중: {video_url}")
        
        # YouTube 객체 생성
        yt = YouTube(video_url)
        
        # 영상 정보 출력
        print(f"📺 제목: {yt.title}")
        print(f"👤 채널: {yt.author}")
        
        # 지정된 해상도의 mp4 스트림 선택
        video_stream = yt.streams.filter(
            res=resolution, 
            file_extension='mp4'
        ).first()
        
        # 지정된 해상도가 없는 경우 사용 가능한 해상도 출력
        if video_stream is None:
            print(f"❌ {resolution} 해상도를 찾을 수 없습니다.")
            print("\n📋 사용 가능한 해상도 목록:")
            
            available_streams = yt.streams.filter(file_extension='mp4')
            resolutions = set()
            
            for stream in available_streams:
                if stream.resolution:
                    resolutions.add(stream.resolution)
            
            for res in sorted(resolutions, key=lambda x: int(x[:-1]), reverse=True):
                stream_info = yt.streams.filter(res=res, file_extension='mp4').first()
                if stream_info:
                    print(f"  🎬 {res} - {stream_info.filesize_mb:.1f}MB")
            
            return False
        
        print(f"✅ {resolution} 해상도 스트림을 찾았습니다!")
        print(f"💾 파일 크기: {video_stream.filesize_mb:.1f}MB")
        
        # 다운로드 폴더 생성
        if not os.path.exists(download_path):
            os.makedirs(download_path)
            print(f"📁 폴더 생성: {download_path}")
        
        # 영상 다운로드
        print(f"⬇️ 다운로드 시작... ({resolution})")
        video_stream.download(download_path)
        print(f"✅ 다운로드 완료!")
        
        return True
        
    except Exception as e:
        print(f"❌ 오류 발생: {str(e)}")
        return False

print("✅ download_with_resolution 함수가 정의되었습니다.")

In [ ]:
# 특정 해상도로 다운로드 실행
# ⚠️ URL을 실제 유튜브 영상 URL로 변경해주세요

video_url = "https://www.youtube.com/watch?v=dQw4w9WgXcQ"
desired_resolution = "720p"  # 원하는 해상도 설정 (720p, 480p, 360p, 1080p 등)

# 다운로드 실행
success = download_with_resolution(video_url, desired_resolution, "./resolution_downloads")

if success:
    print(f"\n🎉 {desired_resolution} 해상도로 다운로드 완료!")
else:
    print(f"\n😞 {desired_resolution} 해상도 다운로드에 실패했습니다.")

## 5️⃣ 오디오만 다운로드 (MP3)

`yt-dlp`를 사용하여 영상에서 오디오만 추출하여 MP3 파일로 저장합니다.

In [ ]:
def download_audio_only(video_url, download_path="./audio", quality="192"):
    """
    유튜브 영상에서 오디오만 MP3 형식으로 다운로드하는 함수
    
    Parameters:
    video_url (str): 다운로드할 유튜브 영상 URL
    download_path (str): 다운로드할 폴더 경로
    quality (str): 오디오 품질 (128, 192, 256, 320 등)
    
    Returns:
    bool: 다운로드 성공 여부
    """
    # 오디오 다운로드 옵션 설정
    ydl_opts = {
        'format': 'bestaudio/best',  # 최고 품질의 오디오 선택
        'outtmpl': os.path.join(download_path, '%(title)s.%(ext)s'),  # 파일명 형식
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',  # 오디오 추출 후처리기
            'preferredcodec': 'mp3',      # MP3 형식으로 변환
            'preferredquality': quality,   # 비트레이트 설정
        }],
        'quiet': False,  # 진행 상황 출력
    }
    
    # 다운로드 폴더 생성
    if not os.path.exists(download_path):
        os.makedirs(download_path)
        print(f"📁 오디오 폴더 생성: {download_path}")
    
    try:
        print(f"🎵 오디오 다운로드 설정:")
        print(f"   📊 품질: {quality}kbps")
        print(f"   📂 저장 경로: {download_path}")
        print(f"   🔗 URL: {video_url}")
        
        # YoutubeDL 객체 생성 및 다운로드 실행
        with YoutubeDL(ydl_opts) as ydl:
            print("\n⬇️ 오디오 다운로드 시작...")
            ydl.download([video_url])
            print("✅ 오디오 다운로드 완료!")
            
        return True
            
    except Exception as e:
        print(f"❌ 오류 발생: {str(e)}")
        return False

print("✅ download_audio_only 함수가 정의되었습니다.")

In [ ]:
# 오디오만 다운로드 실행
# ⚠️ URL을 실제 유튜브 영상 URL로 변경해주세요

video_url = "https://www.youtube.com/watch?v=dQw4w9WgXcQ"
audio_quality = "192"  # 오디오 품질 설정 (128, 192, 256, 320)

# 오디오 다운로드 실행
success = download_audio_only(video_url, "./audio_downloads", audio_quality)

if success:
    print(f"\n🎵 MP3 파일 다운로드 완료! (품질: {audio_quality}kbps)")
else:
    print("\n😞 오디오 다운로드에 실패했습니다.")

## 6️⃣ 영상 정보 확인

다운로드하기 전에 영상의 정보를 미리 확인할 수 있습니다.

In [ ]:
def get_video_info(video_url):
    """
    유튜브 영상의 상세 정보를 확인하는 함수
    
    Parameters:
    video_url (str): 확인할 유튜브 영상 URL
    
    Returns:
    dict: 영상 정보 딕셔너리
    """
    try:
        print(f"🔍 영상 정보 분석 중...")
        print(f"🔗 URL: {video_url}\n")
        
        # YouTube 객체 생성
        yt = YouTube(video_url)
        
        # 기본 정보 출력
        print("📺 기본 정보")
        print("=" * 40)
        print(f"제목: {yt.title}")
        print(f"채널: {yt.author}")
        print(f"조회수: {yt.views:,}회")
        print(f"길이: {yt.length}초 ({yt.length//60}분 {yt.length%60}초)")
        print(f"게시일: {yt.publish_date}")
        
        # 설명 (처음 100자만)
        if yt.description:
            description_preview = yt.description[:100] + "..." if len(yt.description) > 100 else yt.description
            print(f"설명: {description_preview}")
        
        # 사용 가능한 스트림 정보
        print("\n🎬 사용 가능한 비디오 스트림")
        print("=" * 40)
        
        video_streams = yt.streams.filter(file_extension='mp4').order_by('resolution').desc()
        
        if video_streams:
            for stream in video_streams:
                # 파일 크기 정보 (MB 단위)
                size_mb = stream.filesize_mb if hasattr(stream, 'filesize_mb') else 'N/A'
                
                # 스트림 타입 (progressive vs adaptive)
                stream_type = "Progressive" if stream.is_progressive else "Adaptive"
                
                print(f"  📹 해상도: {stream.resolution or 'N/A'} | "
                      f"크기: {size_mb}MB | "
                      f"타입: {stream_type}")
        else:
            print("  ❌ 사용 가능한 MP4 스트림이 없습니다.")
        
        # 오디오 스트림 정보
        print("\n🎵 사용 가능한 오디오 스트림")
        print("=" * 40)
        
        audio_streams = yt.streams.filter(only_audio=True)
        
        if audio_streams:
            for stream in audio_streams[:3]:  # 상위 3개만 표시
                size_mb = stream.filesize_mb if hasattr(stream, 'filesize_mb') else 'N/A'
                print(f"  🎵 코덱: {stream.audio_codec or 'N/A'} | "
                      f"비트레이트: {stream.abr or 'N/A'} | "
                      f"크기: {size_mb}MB")
        else:
            print("  ❌ 사용 가능한 오디오 스트림이 없습니다.")
        
        # 정보를 딕셔너리로 반환
        video_info = {
            'title': yt.title,
            'author': yt.author,
            'views': yt.views,
            'length': yt.length,
            'publish_date': yt.publish_date,
            'description': yt.description,
            'video_streams': len(video_streams),
            'audio_streams': len(audio_streams)
        }
        
        print("\n✅ 영상 정보 분석 완료!")
        return video_info
        
    except Exception as e:
        print(f"❌ 오류 발생: {str(e)}")
        return None

print("✅ get_video_info 함수가 정의되었습니다.")

In [ ]:
# 영상 정보 확인 실행
# ⚠️ URL을 실제 유튜브 영상 URL로 변경해주세요

video_url = "https://www.youtube.com/watch?v=dQw4w9WgXcQ"

# 영상 정보 가져오기
info = get_video_info(video_url)

if info:
    print(f"\n📊 요약: '{info['title']}'는 {info['length']}초 길이의 영상입니다.")
else:
    print("\n😞 영상 정보를 가져올 수 없습니다.")

## 7️⃣ 진행률 표시 다운로드 (yt-dlp)

다운로드 진행률을 실시간으로 확인할 수 있는 고급 다운로드 기능입니다.

In [ ]:
def download_with_progress(video_url, download_path="./downloads", video_quality="best"):
    """
    다운로드 진행률을 표시하면서 영상을 다운로드하는 함수
    
    Parameters:
    video_url (str): 다운로드할 유튜브 영상 URL
    download_path (str): 다운로드할 폴더 경로
    video_quality (str): 비디오 품질 ('best', 'worst', '720p', '480p' 등)
    
    Returns:
    bool: 다운로드 성공 여부
    """
    
    def progress_hook(d):
        """
        다운로드 진행률을 표시하는 콜백 함수
        
        Parameters:
        d (dict): 다운로드 상태 정보
        """
        if d['status'] == 'downloading':
            # 진행률 정보 추출
            percent = d.get('_percent_str', 'N/A')
            speed = d.get('_speed_str', 'N/A')
            eta = d.get('_eta_str', 'N/A')
            
            # 진행률 바 생성 (간단한 버전)
            if '_percent_str' in d:
                try:
                    percent_num = float(d['_percent_str'].replace('%', ''))
                    bar_length = 30
                    filled_length = int(bar_length * percent_num / 100)
                    bar = '█' * filled_length + '░' * (bar_length - filled_length)
                    
                    print(f"\r⬇️ [{bar}] {percent} | 속도: {speed} | 남은시간: {eta}", 
                          end='', flush=True)
                except:
                    print(f"\r⬇️ 진행률: {percent} | 속도: {speed} | 남은시간: {eta}", 
                          end='', flush=True)
            
        elif d['status'] == 'finished':
            filename = os.path.basename(d['filename'])
            print(f"\n✅ 다운로드 완료: {filename}")
        
        elif d['status'] == 'error':
            print(f"\n❌ 다운로드 오류 발생")
    
    # yt-dlp 옵션 설정
    ydl_opts = {
        'outtmpl': os.path.join(download_path, '%(title)s.%(ext)s'),  # 파일명 형식
        'format': f'{video_quality}[ext=mp4]' if video_quality != 'best' else 'best[ext=mp4]',  # 비디오 품질
        'progress_hooks': [progress_hook],  # 진행률 훅 추가
        'noplaylist': True,  # 플레이리스트 다운로드 방지
    }
    
    # 다운로드 폴더 생성
    if not os.path.exists(download_path):
        os.makedirs(download_path)
        print(f"📁 폴더 생성: {download_path}")
    
    try:
        print(f"🎯 다운로드 설정:")
        print(f"   📊 품질: {video_quality}")
        print(f"   📂 저장 경로: {download_path}")
        print(f"   🔗 URL: {video_url}")
        print("\n⬇️ 다운로드 시작...")
        
        # YoutubeDL 객체 생성 및 다운로드 실행
        with YoutubeDL(ydl_opts) as ydl:
            ydl.download([video_url])
        
        print("\n🎉 모든 다운로드가 완료되었습니다!")
        return True
        
    except Exception as e:
        print(f"\n❌ 오류 발생: {str(e)}")
        return False

print("✅ download_with_progress 함수가 정의되었습니다.")

In [ ]:
# 진행률 표시 다운로드 실행
# ⚠️ URL을 실제 유튜브 영상 URL로 변경해주세요

video_url = "https://www.youtube.com/watch?v=dQw4w9WgXcQ"
quality = "720p"  # 원하는 품질 설정 (best, 720p, 480p, 360p 등)

# 진행률 표시 다운로드 실행
success = download_with_progress(video_url, "./progress_downloads", quality)

if success:
    print(f"\n🎊 진행률 표시 다운로드 완료! (품질: {quality})")
else:
    print("\n😞 진행률 표시 다운로드에 실패했습니다.")

## 8️⃣ 다운로드된 파일 관리

다운로드된 파일들을 확인하고 관리하는 유틸리티 함수들입니다.

In [ ]:
def list_downloaded_files(download_path="./downloads"):
    """
    다운로드된 파일 목록을 확인하는 함수
    
    Parameters:
    download_path (str): 확인할 폴더 경로
    
    Returns:
    list: 파일 정보 리스트
    """
    if not os.path.exists(download_path):
        print(f"❌ 폴더가 존재하지 않습니다: {download_path}")
        return []
    
    files = []
    total_size = 0
    
    print(f"📂 다운로드 폴더: {download_path}")
    print("=" * 60)
    
    # 폴더 내 모든 파일 확인
    for filename in os.listdir(download_path):
        filepath = os.path.join(download_path, filename)
        
        if os.path.isfile(filepath):
            # 파일 크기 계산 (MB 단위)
            file_size = os.path.getsize(filepath) / (1024 * 1024)
            total_size += file_size
            
            # 파일 확장자 확인
            file_ext = os.path.splitext(filename)[1].lower()
            
            # 파일 타입 아이콘
            if file_ext in ['.mp4', '.avi', '.mkv', '.mov']:
                icon = "🎬"
            elif file_ext in ['.mp3', '.wav', '.flac', '.aac']:
                icon = "🎵"
            else:
                icon = "📄"
            
            print(f"{icon} {filename}")
            print(f"   📊 크기: {file_size:.1f}MB | 확장자: {file_ext}")
            print()
            
            files.append({
                'name': filename,
                'size_mb': file_size,
                'extension': file_ext,
                'path': filepath
            })
    
    print("=" * 60)
    print(f"📊 총 {len(files)}개 파일 | 전체 크기: {total_size:.1f}MB")
    
    return files

def clean_download_folder(download_path="./downloads", confirm=True):
    """
    다운로드 폴더를 정리하는 함수
    
    Parameters:
    download_path (str): 정리할 폴더 경로
    confirm (bool): 삭제 전 확인 여부
    
    Returns:
    bool: 정리 성공 여부
    """
    if not os.path.exists(download_path):
        print(f"❌ 폴더가 존재하지 않습니다: {download_path}")
        return False
    
    files = os.listdir(download_path)
    
    if not files:
        print(f"✅ 폴더가 이미 비어있습니다: {download_path}")
        return True
    
    print(f"🗑️ {len(files)}개 파일을 삭제할 예정입니다.")
    
    if confirm:
        response = input("정말로 모든 파일을 삭제하시겠습니까? (y/N): ")
        if response.lower() != 'y':
            print("❌ 삭제가 취소되었습니다.")
            return False
    
    try:
        for filename in files:
            filepath = os.path.join(download_path, filename)
            if os.path.isfile(filepath):
                os.remove(filepath)
                print(f"🗑️ 삭제됨: {filename}")
        
        print(f"✅ 폴더 정리 완료: {download_path}")
        return True
        
    except Exception as e:
        print(f"❌ 오류 발생: {str(e)}")
        return False

print("✅ 파일 관리 함수들이 정의되었습니다.")

In [ ]:
# 다운로드된 파일 목록 확인
print("📋 다운로드된 파일 목록 확인\n")

# 각 폴더별로 파일 목록 확인
folders_to_check = ["./downloads", "./batch_downloads", "./resolution_downloads", 
                   "./audio_downloads", "./progress_downloads"]

for folder in folders_to_check:
    if os.path.exists(folder):
        print(f"\n{'='*60}")
        files = list_downloaded_files(folder)
        print(f"{'='*60}")
    else:
        print(f"\n📂 {folder}: 폴더가 존재하지 않습니다.")

## 9️⃣ 종합 실습 및 팁

지금까지 배운 모든 기능을 활용한 종합적인 다운로드 시스템입니다.

In [ ]:
def youtube_downloader_menu():
    """
    사용자 친화적인 유튜브 다운로더 메뉴 시스템
    """
    print("🎥 유튜브 다운로더 v1.0")
    print("=" * 40)
    print("1. 단일 영상 다운로드 (기본)")
    print("2. 특정 해상도로 다운로드")
    print("3. 오디오만 다운로드 (MP3)")
    print("4. 영상 정보 확인")
    print("5. 진행률 표시 다운로드")
    print("6. 다운로드된 파일 목록 확인")
    print("0. 종료")
    print("=" * 40)
    
    while True:
        try:
            choice = input("\n선택하세요 (0-6): ").strip()
            
            if choice == '0':
                print("👋 프로그램을 종료합니다.")
                break
                
            elif choice == '1':
                url = input("유튜브 URL을 입력하세요: ").strip()
                if url:
                    download_youtube_video(url)
                    
            elif choice == '2':
                url = input("유튜브 URL을 입력하세요: ").strip()
                resolution = input("해상도를 입력하세요 (예: 720p): ").strip() or "720p"
                if url:
                    download_with_resolution(url, resolution)
                    
            elif choice == '3':
                url = input("유튜브 URL을 입력하세요: ").strip()
                quality = input("오디오 품질을 입력하세요 (예: 192): ").strip() or "192"
                if url:
                    download_audio_only(url, "./audio", quality)
                    
            elif choice == '4':
                url = input("유튜브 URL을 입력하세요: ").strip()
                if url:
                    get_video_info(url)
                    
            elif choice == '5':
                url = input("유튜브 URL을 입력하세요: ").strip()
                quality = input("비디오 품질을 입력하세요 (예: 720p, best): ").strip() or "best"
                if url:
                    download_with_progress(url, "./downloads", quality)
                    
            elif choice == '6':
                folder = input("확인할 폴더를 입력하세요 (기본: ./downloads): ").strip() or "./downloads"
                list_downloaded_files(folder)
                
            else:
                print("❌ 잘못된 선택입니다. 0-6 사이의 숫자를 입력하세요.")
                
        except KeyboardInterrupt:
            print("\n\n👋 프로그램을 종료합니다.")
            break
        except Exception as e:
            print(f"❌ 오류 발생: {str(e)}")

print("✅ 종합 메뉴 시스템이 준비되었습니다.")
print("💡 아래 셀을 실행하여 대화형 메뉴를 시작하세요!")

In [ ]:
# 대화형 메뉴 실행
# 이 셀을 실행하면 사용자 친화적인 메뉴가 시작됩니다

youtube_downloader_menu()

## 🔧 문제 해결 및 팁

### 자주 발생하는 문제들과 해결 방법

1. **라이브러리 오류**
   - `pytube` 오류 시: `pip install --upgrade pytube` 또는 `pip install pytubefix` 사용
   - `yt-dlp` 오류 시: `pip install --upgrade yt-dlp`

2. **다운로드 실패**
   - URL이 올바른지 확인
   - 인터넷 연결 상태 확인
   - 영상이 비공개/삭제되지 않았는지 확인

3. **권한 오류**
   - 다운로드 폴더의 쓰기 권한 확인
   - 다른 폴더 경로 시도

4. **성능 최적화**
   - 대용량 파일은 충분한 저장 공간 확보
   - 여러 파일 다운로드 시 적절한 대기 시간 설정

### 유용한 팁

- **플레이리스트 다운로드**: `pytube.Playlist` 사용
- **자막 다운로드**: `yt.captions` 활용
- **썸네일 다운로드**: `yt.thumbnail_url` 사용
- **메타데이터 저장**: JSON 파일로 영상 정보 저장

### 법적 고지사항

⚠️ **중요**: 이 도구는 교육 목적으로 제작되었습니다.
- 다운로드한 콘텐츠는 개인적 용도로만 사용하세요
- 저작권을 존중하고 관련 법률을 준수하세요
- 상업적 목적으로 사용하지 마세요
- 콘텐츠 제작자의 권리를 존중하세요

## 🎯 마무리

축하합니다! 🎉 파이썬을 사용한 유튜브 영상 다운로드 시스템을 완성했습니다.

### 배운 내용 요약:
- ✅ `pytube`와 `yt-dlp` 라이브러리 사용법
- ✅ 단일/다중 영상 다운로드
- ✅ 해상도별 다운로드
- ✅ 오디오 추출 (MP3)
- ✅ 영상 정보 확인
- ✅ 진행률 표시
- ✅ 파일 관리 시스템
- ✅ 사용자 친화적 인터페이스

### 다음 단계:
1. GUI 인터페이스 추가 (tkinter, PyQt)
2. 웹 인터페이스 구축 (Flask, Django)
3. 데이터베이스 연동으로 다운로드 기록 관리
4. 자동화 스케줄링 기능
5. 클라우드 저장소 연동

이 노트북을 참고하여 자신만의 유튜브 다운로더를 더욱 발전시켜보세요! 💪

### 📚 추가 학습 자료:
- **공식 문서**: [pytube 문서](https://pytube.io/), [yt-dlp 문서](https://github.com/yt-dlp/yt-dlp)
- **관련 라이브러리**: `requests`, `beautifulsoup4`, `selenium`
- **고급 기능**: 멀티스레딩, 비동기 처리, 에러 핸들링

### 🚀 프로젝트 확장 아이디어:
1. **플레이리스트 전체 다운로드** 기능
2. **자막 다운로드** 및 번역 기능
3. **썸네일 추출** 및 저장
4. **메타데이터 관리** (JSON/CSV 저장)
5. **다운로드 통계** 및 분석 기능
6. **중복 파일 검사** 및 관리
7. **파일 형식 변환** (MP4 → AVI, MP3 → WAV 등)
8. **클라우드 업로드** 자동화
9. **모바일 앱 연동**
10. **AI 기반 콘텐츠 분류**

### ⚡ 성능 최적화 팁:
- **멀티스레딩**으로 동시 다운로드
- **청크 단위 다운로드**로 메모리 효율성 향상
- **캐싱 시스템** 구현으로 중복 요청 방지
- **압축 알고리즘** 적용으로 저장 공간 절약
- **네트워크 대역폭** 최적화
- **에러 재시도** 메커니즘 구현

### 🔒 보안 및 윤리 고려사항:
- **API 키 관리**: 환경 변수 사용
- **사용량 제한**: Rate limiting 구현
- **저작권 준수**: 개인 용도로만 사용
- **데이터 보호**: 사용자 정보 암호화
- **법적 준수**: 각국의 저작권법 확인
- **윤리적 사용**: 콘텐츠 제작자 권리 존중

### 🛠️ 문제 해결 체크리스트:
- [ ] 라이브러리 최신 버전 확인
- [ ] 인터넷 연결 상태 점검
- [ ] URL 유효성 검증
- [ ] 저장 공간 충분성 확인
- [ ] 권한 설정 점검
- [ ] 방화벽/보안 프로그램 확인

### 🎓 학습 성과:
이 프로젝트를 통해 다음과 같은 기술들을 습득했습니다:
- **웹 스크래핑** 기초
- **파일 I/O** 처리
- **예외 처리** 및 에러 핸들링
- **사용자 인터페이스** 설계
- **프로젝트 구조화** 및 모듈화
- **라이브러리 활용** 능력

### 💡 마지막 조언:
- **꾸준한 연습**이 실력 향상의 열쇠입니다
- **오픈소스 프로젝트** 참여로 경험을 쌓아보세요
- **커뮤니티 활동**을 통해 지식을 공유하세요
- **새로운 기술**에 대한 호기심을 유지하세요
- **실제 문제 해결**에 코드를 적용해보세요

이제 여러분은 파이썬을 활용한 멀티미디어 처리의 기초를 마스터했습니다! 🚀

**Happy Coding!** 💻✨

---

*이 노트북이 도움이 되었다면, 다른 사람들과도 공유해보세요! 함께 배우고 성장하는 것이 프로그래밍의 진정한 즐거움입니다.* 🌟